In [ ]:
from input_output import *
results_name, results, target_name, target = choose_results_and_target()

Available datasets:
	Allosteric_Pyramid_Full_Fall22,
	restart_bad_trajs.sh,
	Allosteric_Pyramid_Partial_Fall22,
	AlloSolidCube_X5_4_12Dec22,
	AlloSolidCube_X5_1_12Dec22,
	AlloSolidCube_X3_Singlet_12Dec22,
	check_traj_sizes.sh,
	NoAlloPyramid_14Oct22,
	AlloPyramid_X4_14Mar23,
	jobinfo.txt,
	AlloSolidCube_X1_12Dec22


Enter simulation name:  AlloSolidCube_X5_1_12Dec22


In [57]:
# cutoff = the lower limit for counting clusters as partially assembled forms of the target
# cutoff = 1 -> only count fully assembled clusters
shape = input(f"Enter the export group ({','.join(results.export_groups())}):") 
cutoff = float(input("Input the graph match cutoff: "))
relative = input("Calculate relative yields?") == "True"
overreach = input("Include overreaching graphs in yield calculations?") == "True"
rs = input("Enter resample frequency (in kilosteps, should be >= to the observable print interval):")

Enter the export group (YES_KEY,NO_KEY): NO_KEY
Input the graph match cutoff:  0.075
Calculate relative yields? True
Include overreaching graphs in yield calculations? False
Enter resample frequency (in kilosteps, should be >= to the observable print interval): 1000


In [58]:
df = results.calculateYields(target_name, cutoff, overreach)

YES_KEY_duplicate_0 1 minimal T=0.01 - Max yield: 3.7777777777777777
YES_KEY_duplicate_0 1 minimal T=0.025 - Max yield: 4.0
YES_KEY_duplicate_0 1 minimal T=0.1 - Max yield: 0.2222222222222222
YES_KEY_duplicate_0 1 minimal T=0.05 - Max yield: 3.833333333333333
YES_KEY_duplicate_0 2 minimal T=0.01 - Max yield: 3.9444444444444446
YES_KEY_duplicate_0 2 minimal T=0.025 - Max yield: 4.0
YES_KEY_duplicate_0 2 minimal T=0.1 - Max yield: 0.1111111111111111
YES_KEY_duplicate_0 2 minimal T=0.05 - Max yield: 3.611111111111111
YES_KEY_duplicate_0 3 minimal T=0.01 - Max yield: 3.5555555555555554
YES_KEY_duplicate_0 3 minimal T=0.025 - Max yield: 3.833333333333333
YES_KEY_duplicate_0 3 minimal T=0.1 - Max yield: 0.1111111111111111
YES_KEY_duplicate_0 3 minimal T=0.05 - Max yield: 1.2777777777777777
YES_KEY_duplicate_1 1 minimal T=0.01 - Max yield: 3.555555555555556
YES_KEY_duplicate_1 1 minimal T=0.025 - Max yield: 3.888888888888889
YES_KEY_duplicate_1 1 minimal T=0.1 - Max yield: 0.1111111111111111


In [59]:
# df.set_index(['time','shape','temp','potential','duplicate'], inplace=True)

max_time = df['time'].max()
max_yield = df['yield'].max()

In [60]:
# data = []
# for run_result in results.flat_runs():
#     data += results.annanosecondsalyseClusterYield(run_result, target=target_name)
# df = pd.DataFrame(data)
# # df['time'] /= 10e6 # to make data more readable

rule_data = pd.DataFrame(
    [
        {
            'run_name': results.runs[int(i / len(results.rule))].name,
            'cube_name': results.rule[i % len(results.rule)].name,
            'level': lvl
        }
        for i, lvl in enumerate(itertools.chain.from_iterable([r.cube_type_levels for r in results.runs]))
    ]
)



In [61]:
# resample yields program
picklename = results.root_dir + os.sep + f"target_{target_name}_rs{rs}_cutoff{cutoff}_over{overreach}_rel{relative}.pickle"
if os.path.isfile(picklename):
    df = pd.read_pickle(picklename)
else:
    gb = df.groupby(['shape', "temp", "potential"])
    resampling = gb.resample(rule=f'{rs}us', on='time')
    df = resampling.mean()
    df.drop(labels=['duplicate', 'temp', 'potential'], axis='columns', inplace=True)
    df['yield_min'] = resampling.min()['yield']
    df['yield_max'] = resampling.max()['yield']
    df['yield_stdev'] = resampling.std()['yield']
    df.reset_index(inplace=True)
    df['time'] = pd.to_numeric(df['time'].dt.microseconds)
    df.to_pickle(picklename)

In [62]:
df = df[df['shape'] == shape]

In [63]:
yield_data = [df[df['potential'].copy() == t] for t in df['potential'].unique()]

In [64]:
if (relative):
    for t, yld in zip(results.temperatures(), yield_data):
        yld['yield'] = yld['yield'] / (yld['num_assemblies'] * target['rel_count'])
        yld['yield_min'] = yld['yield_min'] / (yld['num_assemblies'] * target['rel_count'])
        yld['yield_max'] = yld['yield_max'] / (yld['num_assemblies'] * target['rel_count'])
        yld['yield_stdev'] = yld['yield_stdev'] / (yld['num_assemblies'] * target['rel_count'])
    ymax = 1
    x_axis_format = '.0%'
else:
    if (df['num_assemblies'].unique().size > 1):
        print ("Assembly counts are not consistant, scaling may be wonky. You may want to do a relative yield analysis.")
    ymax = df['num_assemblies'].max() * target['rel_count']
    x_axis_format = '.00'

/tmp/ipykernel_3332438/3690572932.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yld['yield'] = yld['yield'] / (yld['num_assemblies'] * target['rel_count'])
/tmp/ipykernel_3332438/3690572932.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yld['yield_min'] = yld['yield_min'] / (yld['num_assemblies'] * target['rel_count'])
/tmp/ipykernel_3332438/3690572932.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [65]:
chartWidth = 800
chartHeight = 250

In [66]:
charts = []
for i, yld in enumerate(yield_data):
    charts.append(alt.Chart(yld, title="Assembly Yields at nt=%f" % results.narrow_types()[i]))
    charts[i] = charts[i].mark_line().encode(
            x=alt.X('time:Q', title="Time (megasteps)", axis=alt.Axis(tickCount=5)),
            y=alt.Y(
                "yield:Q",
                title="Yield",
                scale=alt.Scale(
                    domain=(0, ymax)
                ),
                axis=alt.Axis(format=x_axis_format)
            ),
            color=alt.Color('temp:O', scale=alt.Scale(scheme="accent"), title="Temperature")
    )
    charts[i] = charts[i].properties(width=int(chartWidth / len(results.narrow_types())), height=chartHeight)
    errbars = charts[i].mark_area(opacity=0.25).encode(
        x=alt.X("time:Q", axis=alt.Axis(tickCount=5)),
        y="yield_min:Q",
        y2="yield_max:Q",
        color=alt.Color('temp:O', scale=alt.Scale(scheme="accent"), title="Temperature")
    )
    charts[i] = errbars + charts[i]
        
chart = alt.hconcat(
    alt.Chart(rule_data).mark_bar().encode(
        x=alt.X('run_name', title="Configuration"),
        y=alt.Y('level', title="Level"),
        color=alt.Color('cube_name', title="Type")
    ),
    *charts
)

# ) + alt.Chart(df).mark_errorband(extent='ci', opacity=0.2).encode(
#     x=alt.X('time:Q'),
#     y=alt.Y('yield', title='Yield'),
#     color = alt.Color('shape', scale=alt.Scale())
# )).properties(width=600, height=450).facet(column='potential', row='type').properties(title=sim_name)

chart.properties(title=f"{results.export_name}:{shape} - {target_name} - Cutoff:{cutoff} (Overreach{' ' if overreach else ' not '} allowed)")
# chart.save(sims_root() + os.sep + results.export_name + os.sep + results.export_name + ".html")

alt.HConcatChart(...)

In [67]:
df

,shape,temp,potential,time,yield,num_assemblies,yield_min,yield_max,yield_stdev
0,NO_KEY,0.01,1,0,0.0,4.0,0.0,0.0,0.0
1,NO_KEY,0.01,1,1000,0.0,4.0,0.0,0.0,0.0
2,NO_KEY,0.01,1,2000,0.0,4.0,0.0,0.0,0.0
3,NO_KEY,0.01,1,3000,0.0,4.0,0.0,0.0,0.0
4,NO_KEY,0.01,1,4000,0.0,4.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
1195,NO_KEY,0.10,3,95000,0.0,4.0,0.0,0.0,0.0
1196,NO_KEY,0.10,3,96000,0.0,4.0,0.0,0.0,0.0
1197,NO_KEY,0.10,3,97000,0.0,4.0,0.0,0.0,0.0
1198,NO_KEY,0.10,3,98000,0.0,4.0,0.0,0.0,0.0


In [13]:
np.max(yield_data[1]['yield_max'])

1.0